In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import proplot as plot
from tropCalc import tropCalc
from metpy.units import units

In [ ]:
# iap
pre_1 = pd.read_csv('../data/ozonesondes/data/20190725/20190723_0529.csv')
post_1 = pd.read_csv('../data/ozonesondes/data/20190725/20190725_0634.csv')
post_wrfchem_1 = xr.open_dataset('../data/wrfchem/export/20190725/wrfout_d03_2019-07-25_06:00:00_region_500lnox')
post_wrfchem_2 = xr.open_dataset('../data/wrfchem/export/20200901/wrfout_d04_2020-09-01_05:40:00_region_500lnox')

post_wrfchem_1_00 = xr.open_dataset('../data/wrfchem/export/20190725/20190725_init')
post_wrfchem_2_00 = xr.open_dataset('../data/wrfchem/export/20200901/20200901_init')

# vaisala
pre_2 = pd.read_csv('../data/ozonesondes/data/20200901/20200831_234453.csv')
post_2 = pd.read_csv('../data/ozonesondes/data/20200901/20200901_054434.csv')

Because the temperature sensor of `post_2` sonde didn't work, we have to calculate the pressure from `HeightFromGps`:

## Calculate the tropopause

We need to smooth the data first:

In [ ]:
# https://stackoverflow.com/questions/58133737/savgol-filter-over-dataframe-columns
from scipy.signal import savgol_filter

pre_1_smoothed = pre_1[['PR', 'T']].apply(lambda x: savgol_filter(x,81,1))
post_1_smoothed = post_1[['PR', 'T']].apply(lambda x: savgol_filter(x,81,1))

pre_2_smoothed = pre_2[['P', 'Temp']].apply(lambda x: savgol_filter(x,81,1))
post_2_smoothed = post_2[['PressureFromHeight', 'Temperature']][post_2['Dropping'] == 0].apply(lambda x: savgol_filter(x,81,1))
post_2_smoothed['Temperature'] -= 273.15

In [ ]:
fig, axs = plot.subplots()

l1 = pre_1_smoothed.plot('T', 'PR', ax=axs, label='pre_1')
l2 = post_1_smoothed.plot('T', 'PR', ax=axs, label='post_')
l3 = pre_2_smoothed.plot('Temp', 'P', ax=axs, label='pre_2')
l4 = post_2_smoothed.plot('Temperature', 'PressureFromHeight', ax=axs, label='post_2')

axs.legend([l1, l2, l3, l4], loc='r', ncols=1)

Before smooth:

In [ ]:
lapseC = 2*units.kelvin/units.km
height = False

pre_1_tropo = tropCalc(pre_1['PR'].values*units.hPa,
                       pre_1['T'].values*units.degC,
                       lapseC=lapseC,
                       height=height,
                       method="wmo")

post_1_tropo = tropCalc(post_1['PR'].values*units.hPa,
                        post_1['T'].values*units.degC,
                        lapseC=lapseC,
                        height=height,
                        method="wmo")

pre_2_tropo = tropCalc(pre_2['P'].values*units.hPa,
                       pre_2['Temp'].values*units.degC,
                       lapseC=lapseC,
                       height=height,
                       method="wmo")

pre_1_tropo, post_1_tropo, pre_2_tropo

In [ ]:
pre_1_tropo = tropCalc(pre_1_smoothed['PR'].values*units.hPa,
                       pre_1_smoothed['T'].values*units.degC,
                       lapseC=lapseC,
                       height=height,
                       method="wmo")

post_1_tropo = tropCalc(post_1_smoothed['PR'].values*units.hPa,
                        post_1_smoothed['T'].values*units.degC,
                        lapseC=lapseC,
                        height=height,
                        method="wmo")

pre_2_tropo = tropCalc(pre_2_smoothed['P'].values*units.hPa,
                       pre_2_smoothed['Temp'].values*units.degC,
                       lapseC=lapseC,
                       height=height,
                       method="wmo")

# the data has issue
post_2_smoothed_subset = post_2_smoothed[post_2_smoothed['PressureFromHeight'] < 250]
post_2_tropo = tropCalc(post_2_smoothed_subset['PressureFromHeight'].values*units.hPa,
                       post_2_smoothed_subset['Temperature'].values*units.degC,
                       lapseC=lapseC,
                       height=height,
                       method="wmo")

pre_1_tropo, post_1_tropo, pre_2_tropo, post_2_tropo

Let's get the nearest heights based on the tropopause pressures:

In [ ]:
htropo_pre_1 = pre_1.loc[pre_1['PR'].sub(pre_1_tropo.magnitude).abs().idxmin()]['h']/1e3

htropo_pre_2 = pre_2.loc[pre_2['P'].sub(pre_2_tropo.magnitude).abs().idxmin()]['HeightMSL']/1e3
htropo_post_2 = post_2.loc[post_2['PressureFromHeight'].sub(post_2_tropo.magnitude).abs().idxmin()]['HeightFromGps']/1e3

htropo_pre_1, htropo_pre_2, htropo_post_2

In [ ]:
f, axs = plot.subplots()
ascending = post_2['Dropping'] == 0

# ax = axs[0]
ax = axs
lb_1 = ax.plot(pre_2['ppb'], pre_2['GpsHeightMSL']/1e3, label='Pre. conv')
lb_2 = ax.plot(post_2['ppb'][ascending], post_2['HeightFromGps'][ascending]/1e3, label='Post. conv')
ax.format(ylim=(0,20), xlim=(0, 300), xlabel='O$_3$ (ppbv)', ylabel='Altitude (km)')

# ax=axs[1]
# interp_y = np.arange(0, 20, 0.05)*1e3

f.legend([lb_1, lb_2],
         ncols=1,
         frame=False,
         loc='r',
         fontsize=8)

In [ ]:
def plot_wrfchem(wrfchem, ax, linestyle='-', quantile=True):
    if linestyle == '-':
        label = 'Post/During conv. \n WRF-Chem O$_3$'
    else:
        label = 'Initial WRF-Chem O$_3$'

    wrfchem_legend = ax.plot(wrfchem['medians'],
                         wrfchem['medians'].level,
                         color='green',
                         linestyle=linestyle,
                         label=label)
    
    if quantile:
        ax.fill_betweenx(wrfchem['medians'].level,
                         wrfchem['quantiles'].sel(quantile=0.05).values,
                         wrfchem['quantiles'].sel(quantile=0.95).values,
                         edgecolor='none', facecolor='gray4')

        ax.fill_betweenx(wrfchem['medians'].level,
                         wrfchem['quantiles'].sel(quantile=0.25).values,
                         wrfchem['quantiles'].sel(quantile=0.75).values,
                         edgecolor='none', facecolor='gray6')

#     twiny_1.plot(
#         post_wrfchem_1['medians'],
#         shadedata=post_wrfchem_1['quantiles'].sel(quantile=[0.25, 0.75]), # dark shading
#         fadedata=post_wrfchem_1['quantiles'].sel(quantile=[0.05, 0.95]), # light shading
#         shadelabel='25% - 75%',
#         fadelabel='5% - 95%',
#         color='ocean blue',
#         barzorder=0,
#         boxmarker=False,
#         legend='ll',
#     )

    return wrfchem_legend

In [ ]:
f, axs = plot.subplots(ncols=2, share=0, sharey=3)

# set color
o3_color = 'firebrick'
qv_color = 'royalblue'

# pre-convection
# --- IAP ---
twiny_1 = axs[0].twiny()
pre_o3_1 = twiny_1.plot(pre_1['O3'].values,
                      pre_1['h'].values/1e3,
                      label=r'Pre conv. O$_3$',
                      color=o3_color,
                      linestyle='--')

pre_qv_1 = axs[0].plot(pre_1['QV'].values*1e6,  # ppmv
                       pre_1['h'].values/1e3,
                       label=r'Pre conv. Qv',
                       color=qv_color,
                       linestyle='--')

# --- VAISALA ---
twiny_2 = axs[1].twiny()
surf_above = pre_2['HeightMSL'] > 100
pre_o3_2 = twiny_2.plot(pre_2['ppb'][surf_above].values,
                      pre_2['HeightMSL'][surf_above].values/1e3,
                      label=r'Pre conv. O$_3$',
                      color=o3_color,
                      linestyle='--')

pre_qv_2 = axs[1].plot(pre_2['qv'][surf_above].values*1e6,  # ppmv
                      pre_2['HeightMSL'][surf_above].values/1e3,
                      label=r'Pre conv. Qv',
                      color=qv_color,
                      linestyle='--')

# post_convection
# --- IAP ---
post_o3_1 = twiny_1.plot(post_1['O3'],
                         post_1['h'],
                         label=r'Post/During conv. O$_3$',
                         color=o3_color)

post_qv_1 = axs[0].plot(post_1['QV'],
                        post_1['h'],
                        label=r'Post conv. Qv',
                        color=qv_color)

# --- VAISALA ---
surf_above = post_2['HeightFromGps'] > 100
ascending = post_2['Dropping'] == 0
post_o3_2 = twiny_2.plot(post_2['ppb'][ascending&surf_above].values,
                       post_2['HeightFromGps'][ascending&surf_above].values/1e3,
                       label=r'Post conv. O$_3$',
                       color=o3_color)

# post_qv_2 = axs[1].plot(post_2['qv'][ascending&surf_above].values*1e6,
#                         post_2['HeightFromGps'][ascending&surf_above].values/1e3,
#                         label=r'Post conv. Qv',
#                         color=qv_color)

# --- WRFCHEM ---
wrfchem_1 = plot_wrfchem(post_wrfchem_1, twiny_1)
wrfchem_2 = plot_wrfchem(post_wrfchem_2, twiny_2)
wrfchem_1_00 = plot_wrfchem(post_wrfchem_1_00, twiny_1, linestyle='--', quantile=False)
wrfchem_2_00 = plot_wrfchem(post_wrfchem_2_00, twiny_2, linestyle='--', quantile=False)

# set O3 axis format
twiny_1.format(xcolor='firebrick',
             xlabel=r'O$_3$ (ppbv)',
             xlim=(0, 300),
             ylim=(0, 18),
             ylabel='Altitude (km)')

twiny_2.format(xcolor='firebrick',
             xlabel=r'O$_3$ (ppbv)',
             xlim=(0, 300),
             ylim=(0, 18),
             ylabel='Altitude (km)')

# set Qv axis format
axs.format(abc=True, abcloc='l', abcstyle='(a)',
          xlim=(1, 1e5), xlabel='Qv (ppmv)',
          xformatter='log', xscale='symlog',
          xgrid=False, ygrid=False,
          xcolor=qv_color,
#           collabels=('2019-07-25', '2020-09-01')
          )

# add tropopause line
axs[0].axhline(htropo_pre_1, color='gray6', linestyle='--')
pre_tropo = axs[1].axhline(htropo_pre_2, color='gray6', linestyle='--', label='Pre conv. tropopause')
post_tropo = axs[1].axhline(htropo_post_2, color='gray6', linestyle='-', label='Post conv. tropopause')


# plot legend
f.legend((pre_tropo, post_tropo)+pre_o3_1+post_o3_1+\
         pre_qv_1+post_qv_1+wrfchem_1_00+wrfchem_1,
         ncols=2,
         frame=False,
         loc='b',
         fontsize=8)

f.savefig('../figures/sonde_profile.pdf')